In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

<IPython.core.display.Javascript object>

In [3]:
def load_data(spreadsheet, credentials="key.json"):

    # Authorize a client to communicate with the Google API
    scope = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/drive",
    ]
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)
    client = gspread.authorize(creds)

    # Load the Google sheet as a dictionary
    sheet = client.open(spreadsheet).sheet1
    sheet = sheet.get_all_records()

    # Return pandas dataframe
    df = pd.DataFrame(sheet)
    return df

<IPython.core.display.Javascript object>

In [4]:
pizza = load_data("2020/02/06")
pizza["Price"] = pizza["Price"].replace("[\$,]", "", regex=True).astype(float)
pizza

,Item,Price,People
0,Philly Steak + Onion,7.99,"Sebastian, Vivek"
1,Lava,3.00,Sebastian
2,Lava,3.00,Sebastian
3,Lava,3.00,"Sebastian, Yovel"
4,Cheese Pizza,5.99,"Fallon, Yoobin"
5,Meatzza,5.99,"Stephen, Owen"
6,Cheezy Bread,5.99,"Lucas, Vivek"
7,Garlic Dipping Sauce,0.50,Sebastian
8,Taxes,2.31,All
9,Tip,6.56,All


<IPython.core.display.Javascript object>

In [5]:
def _get_attendees(df, people_col):

    attendees = []

    for people in df[people_col]:
        if people == "All":
            continue

        for person in people.split(","):
            person = person.lstrip()
            if person in attendees:
                continue
            else:
                attendees.append(person)

    return attendees

<IPython.core.display.Javascript object>

In [6]:
def _get_individual_cost(df, name, people_col, price_col):

    subset = df[df[people_col].str.contains(name)]

    cost = 0

    for _, (_, price, people) in subset.iterrows():
        n_participants = len(people.split(","))
        cost += price / n_participants

    return cost

<IPython.core.display.Javascript object>

In [7]:
def _get_group_cost(df, cost_dict, people_col, price_col):

    # Get individual and group data sets
    individual = df.query(f"{people_col} != 'All'")
    group = df.query(f"{people_col} == 'All'")

    # Get number of attendees
    n_attendees = len(cost_dict)

    # Get total individual costs (the subtotal) and additional costs
    subtotal = sum(individual[price_col])
    additional = sum(group[price_col])

    # Proportionally add additional costs
    for person, cost in cost_dict.items():
        proportion_of_subtotal = cost / subtotal
        cost_dict[person] += proportion_of_subtotal * additional

    return cost_dict

<IPython.core.display.Javascript object>

In [8]:
def _make_pretty(cost_dict):
    for person, cost in cost_dict.items():
        cost_dict[person] = round(cost, 2)
    return cost_dict

<IPython.core.display.Javascript object>

In [9]:
def main(df, people_col, price_col):

    # Get attendees
    attendees = _get_attendees(df, people_col)

    # Estimate prelimary cost dictionary
    cost_dict = dict()
    for attendee in attendees:
        cost = _get_individual_cost(df, attendee, people_col, price_col)
        cost_dict[attendee] = cost

    # Include delivery fee, tax, and tip
    cost_dict = _get_group_cost(df, cost_dict, people_col, price_col)
    cost_dict = _make_pretty(cost_dict)

    return cost_dict

<IPython.core.display.Javascript object>

In [10]:
main(pizza, "People", "Price")

{'Sebastian': 16.01,
 'Vivek': 9.33,
 'Yovel': 2.0,
 'Fallon': 4.0,
 'Yoobin': 4.0,
 'Stephen': 4.0,
 'Owen': 4.0,
 'Lucas': 4.0}

<IPython.core.display.Javascript object>